In [ ]:
%load_ext autoreload

%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os
print(os.getcwd())
print(os.chdir('/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/'))
print(os.getcwd())

import sys
sys.path.append('./src')

In [ ]:
from prompt import time_series_to_image, LIMIT_PROMPT, PROMPT, time_series_to_str
from utils import view_base64_image, display_messages, collect_results
from utils import interval_to_vector, vector_to_interval
from data.synthetic import SyntheticDataset
import pandas as pd
import numpy as np
import re
from sklearn.metrics import precision_score, recall_score
from affiliation.generics import convert_vector_to_events
from affiliation.metrics import pr_from_events
from utils import compute_metrics

import glob

In [ ]:

def extract_anomaly_sequences(text):
    pattern = r"anomaly data\s*\d+:\s*([-\d.\s]+)"
    matches = re.findall(pattern, text)
    
    anomaly_sequences = []
    for match in matches:
        # 매치된 문자열을 공백 기준으로 분리하고 float형으로 변환
        numbers = [float(num) for num in match.strip().split()]
        anomaly_sequences.append(numbers)
    
    return anomaly_sequences

In [ ]:
# 입력이 dictionary의list임.
def average_dict_values(dict_list):
    sums = {}
    counts = {}

    # list iter
    for d in dict_list:
        # dixt iter
        for key, value in d.items():
            sums[key] = sums.get(key, 0) + value
            counts[key] = counts.get(key, 0) + 1
            
    # 각 키에 대해 평균 계산: 합계 / 등장횟수
    averages = {key: sums[key] / counts[key] for key in sums}
    return averages

## Evaluate

In [ ]:
# data_name!!
data_name_list = ["freq", "point", "range", "trend"]

# Initialize dictionaries instead of lists
anomaly_indexs = {data_name: {} for data_name in data_name_list}
normal_indexs = {data_name: {} for data_name in data_name_list}
eval_results = {data_name: {} for data_name in data_name_list}

for data_name in data_name_list:

    # gt 때문에 loading함.
    data_dir = f'data/synthetic/{data_name}/eval/'
    eval_dataset = SyntheticDataset(data_dir)
    eval_dataset.load()

    model_name = 'gemini-1.5-flash' #'gpt-4o-mini' #'gemini-1.5-flash'
    jsonl_list = glob.glob(f'./results/synthetic/{data_name}/{model_name}/*')

    # eval 할 것들.
    # jsonl 파일 하나에 대해서 분해를 할 수 있다.

    for json_name in jsonl_list:
        anomaly_indexs[data_name][json_name] = []
        normal_indexs[data_name][json_name] = []
        eval_results[data_name][json_name] = []

        result_df = pd.read_json(json_name, lines=True)

        result_df['input_prompt'] = result_df['request'].apply(lambda x:x['messages'][0]['content'])
        result_df['eval_idx'] = result_df['custom_id'].apply(lambda x:int(x.split('_')[-1]))
        result_df['incontext'] = result_df['input_prompt'].apply(lambda x:extract_anomaly_sequences(x))

        raw_result = collect_results(f'./results/synthetic/{data_name}/{model_name}/')
        key_name = json_name.split('/')[-1].replace('.jsonl', '')
        target_res = raw_result[f' ({key_name})']

        for i, eval_data in enumerate(eval_dataset):
            sample_ano_loc, sample_series = eval_data


            if sample_ano_loc.shape[-1] == 0:
                normal_indexs[data_name][json_name].append(i)
                dummy_res = {'precision': 0,
                'recall': 0,
                'f1': 0,
                'affi precision': 0,
                'affi recall': 0,
                'affi f1': 0}
                eval_results[data_name][json_name].append(dummy_res)
                continue
            
            anomaly_indexs[data_name][json_name].append(i)
                
            res_idx = result_df[result_df.eval_idx == i+1].index[0] # index가 0부터 399까지
            #print(res_idx)
            pred_vector = target_res[res_idx]
            #print(pred_vector)
            gt = interval_to_vector(sample_ano_loc[0], end=1000)
            #print(gt)
            our_metric = compute_metrics(gt, pred_vector)
            eval_results[data_name][json_name].append(our_metric)

            if i % 20 == 0:
                print(i, sample_ano_loc.shape)

## Checking evaluate results

In [ ]:
data_name_list = ["freq", "point", "range", "trend"]

model_name = 'gemini-1.5-flash' #'gpt-4o-mini' #'gemini-1.5-flash'

for data_name in data_name_list:
    
    jsonl_list = glob.glob(f'./results/synthetic/{data_name}/{model_name}/*')
    for jsonl in jsonl_list:
        total_result = average_dict_values(eval_results[data_name][jsonl])
        print(f"{data_name}_{jsonl}")
        for k, v in total_result.items():
            print(f"{k}: {v*100:.2f}")
        print("\n\n")

## Making a table with all data (including data without anomalies)

In [ ]:
import glob
import os

data_name_list = ["freq", "point", "range", "trend"]
model_name = 'gemini-1.5-flash'  # 'gpt-4o-mini' #'gemini-1.5-flash'

# 결과를 저장할 리스트 생성
table_data = []

# 헤더 정의
headers = ["Data Type", "Model Type", "Precision", "Recall", "F1", "Affi Precision", "Affi Recall", "Affi F1"]

# 각 열의 최대 너비 계산을 위한 딕셔너리
col_width = {header: len(header) for header in headers}

# 데이터 수집
for data_name in data_name_list:
    jsonl_list = glob.glob(f'./results/synthetic/{data_name}/{model_name}/*')
    
    for jsonl in jsonl_list:
        # 파일 이름만 추출 (경로 제외)
        model_type = os.path.basename(jsonl).replace('.jsonl', '')
        
        # 결과 계산
        total_result = average_dict_values(eval_results[data_name][jsonl])
        
        # 행 데이터 생성
        row = [
            data_name,
            model_type,
            f"{total_result['precision']*100:.2f}",
            f"{total_result['recall']*100:.2f}",
            f"{total_result['f1']*100:.2f}",
            f"{total_result['affi precision']*100:.2f}",
            f"{total_result['affi recall']*100:.2f}",
            f"{total_result['affi f1']*100:.2f}"
        ]
        
        # 열 너비 업데이트
        for i, value in enumerate(row):
            col_width[headers[i]] = max(col_width[headers[i]], len(value))
            
        table_data.append(row)

# 테이블 출력 형식 정의
format_str = " | ".join([f"{{:<{col_width[header]}}}" for header in headers])
separator = "-+-".join(["-" * col_width[header] for header in headers])

# 헤더 출력
print(format_str.format(*headers))
print(separator)

# 데이터 행 출력
for row in table_data:
    print(format_str.format(*row))

## Making a table with only data that has anomalies

In [ ]:
import glob
import os

data_name_list = ["freq", "point", "range", "trend"]
model_name = 'gemini-1.5-flash'  # 'gpt-4o-mini' #'gemini-1.5-flash'

# 결과를 저장할 리스트 생성
table_data = []

# 헤더 정의
headers = ["Data Type", "Model Type", "Precision", "Recall", "F1", "Affi Precision", "Affi Recall", "Affi F1"]

# 각 열의 최대 너비 계산을 위한 딕셔너리
col_width = {header: len(header) for header in headers}

# 데이터 수집
for data_name in data_name_list:
    jsonl_list = glob.glob(f'./results/synthetic/{data_name}/{model_name}/*')
    
    for jsonl in jsonl_list:
        # 파일 이름만 추출 (경로 제외)
        model_type = os.path.basename(jsonl).replace('.jsonl', '')
        
        # 결과 계산
        anomaly_results = [eval_results[data_name][jsonl][a_idx] for a_idx in anomaly_indexs[data_name][jsonl]]
        total_result = average_dict_values(anomaly_results)
        
        # 행 데이터 생성
        row = [
            data_name,
            model_type,
            f"{total_result['precision']*100:.2f}",
            f"{total_result['recall']*100:.2f}",
            f"{total_result['f1']*100:.2f}",
            f"{total_result['affi precision']*100:.2f}",
            f"{total_result['affi recall']*100:.2f}",
            f"{total_result['affi f1']*100:.2f}"
        ]
        
        # 열 너비 업데이트
        for i, value in enumerate(row):
            col_width[headers[i]] = max(col_width[headers[i]], len(value))
            
        table_data.append(row)

# 테이블 출력 형식 정의
format_str = " | ".join([f"{{:<{col_width[header]}}}" for header in headers])
separator = "-+-".join(["-" * col_width[header] for header in headers])

# 헤더 출력
print(format_str.format(*headers))
print(separator)

# 데이터 행 출력
for row in table_data:
    print(format_str.format(*row))